In [ ]:
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../../")

import textwrap

from utils.utils import normalize_cols
from utils_sheets import save_data_in_sheets
import base64
import requests
import json
import io




from IPython.display import display, Audio

import random
import datetime
import glob


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import basedosdados as bd


bd.config.billing_project_id = 'rj-escritorio-dev'
bd.config.from_file = True
bd.__version__


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from PIL import Image, ImageDraw, ImageFont


- storage images: https://console.cloud.google.com/storage/browser/datario-public/flooding_detection?project=datario
- imagens figma: https://www.figma.com/file/Qv89NLopXS60Lqf3XfTZiN/Untitled?type=design&node-id=3-44&mode=design&t=3a4g8D4QLiDQ8f3i-0
- langchain ref: https://python.langchain.com/docs/integrations/platforms/google


In [ ]:
def get_image_link_from_storage():
    dataset_id = "flooding_detection"
    table_id = "classified_images"
    st = bd.Storage(dataset_id=dataset_id, table_id=table_id)
    blobs = (
        st.client["storage_staging"]
        .bucket("datario-public")
        .list_blobs(prefix=f"{dataset_id}/{table_id}")
    )

    url_list = []
    for blob in blobs:
        url = str(blob.public_url)
        if "." in url.split("/")[-1]:
            url_list.append(url)
    return url_list


def get_urls_and_labels():
    urls = get_image_link_from_storage()
    labels = [
        {"path": "images_with_label/flood", "label": True, "object": "alagamento"},
        {"path": "images_with_label/no_flood", "label": False, "object": "alagamento"},
    ]
    data = []
    for url in urls:
        for item in labels:
            if item.get("path") in str(url):
                data.append(
                    {
                        "object": item.get("object"),
                        "label": item.get("label"),
                        "image_url": url,
                    }
                )
    return pd.DataFrame(data)


def balance_and_sample(df, N):
    # Get the minimum count of the two labels
    min_count = min(df["label"].value_counts())

    # Balance the DataFrame
    df_balanced = pd.concat(
        [df[df["label"] == True].head(min_count), df[df["label"] == False].head(min_count)]
    )
    df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
    # Sample N rows
    if N > len(df_balanced):
        print(
            f"Requested number of samples ({N}) is more than the available balanced dataset size ({len(df_balanced)})."
        )
        return df_balanced
    return df_balanced.head(N)


# OpenAI API Key


def get_image_from_url(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content))
    img.thumbnail((640, 480))
    return img


def get_ai_label(response):
    if response.get("error"):
        return "Error"
    else:
        # r = response['choices'][0]['message']['content']
        json_string = r.replace("```json\n", "").replace("\n```", "")
        json_object = json.loads(json_string)
        return json_object["label"]


def get_ai_label_gemini(response):
    if type(response) == tuple:
        response = response[0]
    json_string = str(response).replace("```json\n", "").replace("\n```", "")
    json_object = json.loads(json_string)
    return json_object["label"]


def gemini_pro_vision_classify_image(image):
    prompt = """
            "You are an expert flooding detector. You are
            given a image. You must detect if there is flooding in the image. The output MUST
            be a JSON object with a boolean value for the key ""label"". If you don't
            know what to anwser, you can set the key ""label"" as false. Example:
            {
                ""label"": true
            }"
    """
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        model = genai.GenerativeModel("gemini-pro-vision")
        responses = model.generate_content(
            contents=[prompt, image],
            generation_config={
                "max_output_tokens": 2048,
                "temperature": 0.4,
                "top_p": 1,
                "top_k": 32,
            },
            stream=True,
        )
        responses.resolve()
        return (responses.text, True, None)
    except Exception as e:
        return (' {\n  "label": false\n}', False, str(e))

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI, chat_models


def get_content():
    image_classificacao = "https://storage.googleapis.com/datario-public/flooding_detection/prompt_images/exemplos_classificacao.png"
    image_exemplo = "https://storage.googleapis.com/datario-public/flooding_detection/prompt_images/bolsao_dagua.png"
    content = [
        {
            "type": "text",
            "text": """
                You are an expert flooding detector. 
                Given an image, you must detect the following categories otherwise return null:
                 - lamina_dagua: Area with water up to 15 cm, possible to identify a pedestrian with water up to their ankles.
                 - bolsao_dagua: Accumulation of water between 15 and 30 cm, small vehicle with water reaching a quarter of the height of the wheels.
                 - alagamento: Water between 30 and 50 cm, vehicles partially submerged, with water up to half the wheels.
                 - null: None of the above descriptions
                
                Using this examples from the image:
                
            """,
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": image_classificacao},
        # example
        {
            "type": "text",
            "text": """
                INPUT
                A CCTV image.

                OUTPUT
                The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

                ```json
                {
                    "object": string  // answer to the user's question
                    "explanation": string  // Explanation for the object and object classification
                }
                ```
                
                Example:
                
                OUTPUT:
                ```json
                {
                    "object": "bolsao_dagua",
                    "explanation": "Because ..."
                }
                ```
                INPUT:
                
            """,
        },
        {"type": "image_url", "image_url": image_exemplo},
        {
            "type": "text",
            "text": "Now classify the image bellow:",
        },
    ]

    for d in content:
        for key, value in d.items():
            d[key] = textwrap.dedent(value)

    return content


def gemini_pro_vision_langchain(
    image_url, content, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro-vision",
        google_api_key=GOOGLE_API_KEY,
        max_output_token=max_output_token,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )

    message = HumanMessage(content=content + [{"type": "image_url", "image_url": image_url}])
    return llm.invoke([message])


def get_parser():
    from langchain.output_parsers import StructuredOutputParser, ResponseSchema

    response_schemas = [
        ResponseSchema(name="object", description="answer to the user's question"),
        ResponseSchema(
            name="explanation", description="Explanation for the object and object classification"
        ),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    return output_parser

In [ ]:
def get_urls_from_path(url_path):
    urls = get_image_link_from_storage()
    data = []
    for url in urls:
        if url_path in str(url):
            data.append(
                {
                    "image_url": url,
                }
            )
    return pd.DataFrame(data)


df = get_urls_from_path(url_path="images_predicted_as_flood")


experiment_name = "test"
experiment_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
content = get_content()
max_output_token = 300
temperature = 0.4
top_k = 32
top_p = 1

In [ ]:
from pathlib import Path

predictions_path = Path(f"./data/predictions/{experiment_name}__{experiment_datetime}.csv")

if predictions_path.exists():
    predictions = pd.read_csv(predictions_path)
    predictions_list = predictions["image_url"].tolist()
else:
    predictions_list = []

output_parser = get_parser()


for index, row in df.iterrows():
    image_url = row["image_url"]
    if image_url not in predictions_list:
        response = gemini_pro_vision_langchain(
            image_url=image_url,
            content=content,
            max_output_token=max_output_token,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )
        response_parsed = output_parser.parse(response.content)
        response_parsed["image_url"] = image_url

        print(f"{index} - {len(df)}")
        print(json.dumps(response_parsed, indent=4))
        display(get_image_from_url(image_url))

        save_data_in_sheets(
            save_data=True,
            data={
                "content": get_content(),
                "max_output_token": max_output_token,
                "temperature": temperature,
                "top_k": top_k,
                "top_p": top_p,
                "experiment_name": experiment_name,
                "experiment_datetime": experiment_datetime,
                "true_object": "",
                "object": response_parsed.get("object", ""),
                "explanation": response_parsed.get("explanation", ""),
                "image_url": image_url,
                "image": f'=IMAGE("{image_url}")',
            },
            data_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=436224340",
            content_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1779223884",
        )

        pd.DataFrame([response_parsed]).to_csv(
            path_or_buf=predictions_path,
            index=False,
            header=not predictions_path.exists(),
            mode="a",
        )
    else:
        print(f"{index} - {len(df)}: already predicted")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
import seaborn as sns


true_values = df_final["flood"].tolist()
predicted_values = df_final["ai_label"].tolist()

# Calculate metrics
accuracy = accuracy_score(true_values, predicted_values)
precision = precision_score(true_values, predicted_values, pos_label=True)
recall = recall_score(true_values, predicted_values, pos_label=True)
f1 = f1_score(true_values, predicted_values, pos_label=True)
conf_matrix = confusion_matrix(true_values, predicted_values)

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


cm = confusion_matrix(true_values, predicted_values)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["False", "True"],
    yticklabels=["False", "True"],
)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()


# resize_factor = 3
# imgs = 10
# time = 1:00
# Accuracy: 0.7
# Precision: 0.6666666666666666
# Recall: 0.5
# F1 Score: 0.5714285714285715

# resize_factor = 1
# imgs = 10
# time = 1:15
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75


# resize_factor = 5
# imgs = 10
# time = 1:20
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75

In [ ]:
df_final["miss"] = np.where(df_final["flood"] == df_final["ai_label"], False, True)
mask = df_final["miss"] == True
miss = df_final[mask]
miss_imgs = miss["base64"].tolist()
miss_ai_labels = miss["ai_label"].tolist()


for base64_image, ai_label in zip(miss_imgs, miss_ai_labels):
    print(f"AI classyfy as: {ai_label}")
    display_img(base64_image)